# Hybrid Based Filtering
- Hybrid Based Filtering (HBF) adalah salah satu metode sistem rekomendasi yang mengkombinasikan antara Collaborative Filtering (CF) dan Content Based Filtering (CBF).
- Urutan pengerjaan saya adalah pertama saya menggunakan CBF untuk mendapatkan film rekomendasi awal lalu hasil dari film tersebut menjadi input untuk algoritma CF.

## Case Story
- Rangga, seorang data scientist di netclick.com mendapatkan tambahan tugas baru dari managernya yaitu bos Asep.
- Ternyata tetangganya bos Asep yaitu Ucup kurang puas dengan rekomendasi film dari Rangga sebelumnya, lalu asep menyuruh Rangga untuk mencoba menggunakan algoritma Hybrid Filtering untuk membuat rekomendasi ulang untuk Ucup
- Ucup User ID adalah 352

# Content Based Filtering

## Load Library

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

## Load Datasets
- Dataset yang digunakan berasal dari MovieLens dengan 2 datasets yang akan kita gunakan yaitu data ratings dan movies.

In [3]:
movies = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\movies.csv')
ratings = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\ratings.csv')

## Data Cleaning

In [4]:
#Menghapus tahun di title 
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')

#Menghapus whitespace
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [5]:
#Split genres karena akan digunakan di algoritma CBF
movies['genres'] = movies['genres'].str.split('|')

In [6]:
#Drop kolom timestamp
ratings = ratings.drop(columns=['timestamp'])

In [7]:
def rekomendasi_film_content(user_id):
    #Melakukan One Hot Encoding untuk dataframe baru
    #Kita copy dulu
    new_movies = movies.copy()
    #Kita looping untuk setiap row di df, lalu looping lagi di genres list dan taroh list tsb ke dalam kolom
    for index, row in movies.iterrows():
        for genre in row['genres']:
            new_movies.at[index, genre] = 1    
    #Yang tidak termasuk genrenya ke dalam list maka kita diisi dengan 0
    new_movies = new_movies.fillna(0)
    
    #Kita membuat movie input dari user
    rating_grouping = ratings.groupby('userId')
    user_id_film = rating_grouping.get_group(user_id).head(5)
    
    #Kemudian melakukan One Hot Encoding untuk input movie user
    user_movie_encoding = new_movies[new_movies['movieId'].isin(user_id_film['movieId'].tolist())]
    #Reset index
    user_movie_encoding = user_movie_encoding.reset_index(drop=True)
    #Drop unused columns
    user_movie_encoding = user_movie_encoding.drop(columns=['movieId', 'title', 'genres'])
    
    #Mencari tahu ucup profile berdasarkan genrenya
    #Menggunakan dot product untuk mendapatkan bobot dari setiap genre
    user_profile = user_movie_encoding.transpose().dot(user_id_film['rating'].reset_index(drop=True))
    
    #Melakukan Rekomendasi Film
    new_movies.set_index(new_movies['movieId'], inplace=True)
    new_movies = new_movies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
    #Multiply the genres by the weights and then take the weighted average
    rekomendasi_film = ((new_movies*user_profile).sum(axis=1))/(user_profile.sum())
    #Urutkan berdasarkan nilai weughted average tertinggti
    rekomendasi_film = rekomendasi_film.sort_values(ascending=False)
    #Rekomendasi film final untuk User
    rekomendasi_film_final = movies.loc[movies['movieId'].isin(rekomendasi_film.head(10).keys())]
    
    return rekomendasi_film_final

In [8]:
user = int(input('Masukan User Id: '))
rekomendasi_film_content(user)

Masukan User Id: 352


,movieId,title,genres
167,198,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]"
1330,1799,Suicide Kings,"[Comedy, Crime, Drama, Mystery, Thriller]"
4940,7445,Man on Fire,"[Action, Crime, Drama, Mystery, Thriller]"
5327,8860,Cellular,"[Action, Crime, Drama, Mystery, Thriller]"
5556,26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M..."
5672,27674,11:14,"[Comedy, Crime, Drama, Mystery, Thriller]"
5808,31921,"Seven-Per-Cent Solution, The","[Adventure, Comedy, Crime, Drama, Mystery, Thr..."
7372,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill..."
7441,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film..."
9411,165347,Jack Reacher: Never Go Back,"[Action, Crime, Drama, Mystery, Thriller]"


In [9]:
input_ke_CF = rekomendasi_film_content(user).head(5)

***

***

# Collaborative Filtering

In [11]:
movies = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\movies.csv', encoding='latin1')
ratings = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\ratings.csv')

## Data Cleaning

In [12]:
#Menghapus kolom yang tidak digunakan di df movies
movies.drop(columns=['genres'], inplace=True)

In [13]:
#Menghapus tahun di title
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')

#Menghapus whitesoace
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [14]:
#Menghapus kolom timestamp di df ratings
ratings.drop(columns=['timestamp'], inplace=True)

In [15]:
def rekomendasi_film(list_film):
    #Indexing dan Memberikan rating pada list_film
    list_film.reset_index(drop=True, inplace=True)
    #Rating input
    rating = ratings[ratings['movieId'].isin(input_ke_CF['movieId'].tolist())].groupby('movieId').rating.mean().reset_index(drop=True)
    list_film['rating'] = rating
    
    #Mencari user yang sudah melihat input movie
    list_movie = list_film
    set_user = ratings[ratings['movieId'].isin(list_movie['movieId'].tolist())]
    set_user_grouping = set_user.groupby(['userId'])
    
    #Lalu saya akan melakukan sorting data user berdasarkan kesamaan film input dan film yang sudah ditonton user.
    set_user_grouping = sorted(set_user_grouping,  key=lambda x: len(x[1]), reverse=True)
    
    #Membuat dictionary dimana user id adalah key dan value adalah koefisien pearson
    pearson_dict = {}

    #Looping di set_user_grouping
    for name, group in set_user_grouping:
    
        #Kita sorting dulu agar tidak mixed up
        group = group.sort_values(by='movieId')
        list_movie = list_movie.sort_values(by='movieId')
    
        #cari jumlah dari data 
        n_value = len(group)

        #cari review scores movies yang disesuaikan dengan film yang sudah ditonton cinta
        df_sementara = list_movie[list_movie['movieId'].isin(group['movieId'].tolist())]

        #Simpan rating menjadi bentuk list
        rating_list = df_sementara['rating'].tolist()

        #Masukan juga reviews satu satu dari user yang ada di set_user_grouping kedalam bentuk list
        grup_list = group['rating'].tolist()

        #Lalu lakukan perhitungan untuk pearson correlation antara 2 users, cinta dan current user = x and y
        Sxx = sum([i**2 for i in rating_list]) - pow(sum(rating_list),2)/float(n_value)
        Syy = sum([i**2 for i in grup_list]) - pow(sum(grup_list),2)/float(n_value)
        Sxy = sum( i*j for i, j in zip(rating_list, grup_list)) - sum(rating_list)*sum(grup_list)/float(n_value)

        #Checking apakah penyebutnya 0 atau tidak, kalo tidak maka bisa dibagi, kalo tidak maka 0.
        if Sxx != 0 and Syy != 0:
            pearson_dict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearson_dict[name] = 0
            
    #Organisasi pearson dictionary ke dalam dataframe
    df_pearson = pd.DataFrame.from_dict(pearson_dict, orient='index')
    df_pearson.columns = ['similarityIndex']
    df_pearson['userId'] = df_pearson.index
    df_pearson.index = range(len(df_pearson))
    
    #Kita cari top 50 aja yang similarity indexnya paling mendekati dengan input user
    top_50=df_pearson.sort_values(by='similarityIndex', ascending=False)[0:50]
    
    #Tahap Rekomendasi Film
    
    #Pertama, merge dulu ratings dan top50
    top_50_new = top_50.merge(ratings, left_on='userId', right_on='userId', how='inner')
    
    #Kedua, Kalikan similarity index dan rating
    top_50_new['bobot_rating'] = top_50_new['similarityIndex']*top_50_new['rating']
    
    #Ketiga, groupby berdasarkan movieID dan hanya mengambil kolom similarity index dan bobot_rating
    bobot_df = top_50_new.groupby('movieId').sum()[['similarityIndex','bobot_rating']]
    bobot_df.columns = ['similarity_index_total','bobot_rating_total']
    
    #Keempat, kita bagi bobot rating total dengan similarity index total
    bobot_df['final_recommendation_score'] = bobot_df['bobot_rating_total']/bobot_df['similarity_index_total']
    bobot_df['movieId'] = bobot_df.index
    
    #Kelima, kita sorting dari nilai recommendation score yang paling tinggi
    final_rekomenadasi_film = bobot_df.sort_values(by=['final_recommendation_score'], ascending=False)
    final_rekomenadasi_film.drop(columns=['similarity_index_total', 'bobot_rating_total'], inplace=True)
    
    final_movie_rekomendasi = movies.loc[movies['movieId'].isin(final_rekomenadasi_film.head(10)['movieId'].tolist())]
    return final_movie_rekomendasi

In [16]:
rekomendasi_film(input_ke_CF)

,movieId,title
553,663,Kids in the Hall: Brain Candy
4015,5673,Punch-Drunk Love
4054,5772,My Dinner with AndrÃ©
4083,5833,Dog Soldiers
4119,5909,Visitor Q (Bizita Q)
7302,76093,How to Train Your Dragon
7465,81834,Harry Potter and the Deathly Hallows: Part 1
7579,86142,13 Assassins (JÃ»san-nin no shikaku)
7611,86898,"Tree of Life, The"
7645,88129,Drive


- Jadi setelah melakukan analisis, Rangga melaporkan hasilnya kepada bosnya yaitu top 10 film yang wajib ditonton oleh Ucup:
    1. Kids in the Hall: Brain Candy
    2. Punch-Drunk Love
    3. My Dinner with AndrÃ©
    4. Dog Soldiers
    5. Visitor Q (Bizita Q)
    6. How to Train Your Dragon
    7. Harry Potter and the Deathly Hallows: Part 1
    8. 13 Assassins (JÃ»san-nin no shikaku)
    9. Tree of Life, The
    10. Drive